# Task 2 - Neural network tests

In [1]:
import numpy as np

# Import neural network
import sys
sys.path.insert(0, '../src')

from Task_2.activation_layer import ActivationLayer
from Task_2.adam_optimizer import AdamOptimizer
from Task_2.layer import Layer
from Task_2.neural_network import NeuralNetwork

## Dropout

In [2]:
## Testing the dropout 
l = Layer(10, dropout_percentile = 0.5, activation_function = None)
l.w = np.random.randint(0, 10, (10, 5))

# Fix variables to 0
l.b = np.zeros((1,10))
l.b_moment1 = np.zeros_like(l.b)
l.b_moment2 = np.zeros_like(l.b)
l.w_moment1 = np.zeros_like(l.w)
l.w_moment2 = np.zeros_like(l.w)

l.dropout([0,1,2])
previous = [0,1,2]
w_test = l.w
l.w = 99*np.ones_like(w_test)
l.reset_matrix(previous)
l.w

# Check the 99 in the weigth matrix

array([[ 6,  3,  6,  5,  4],
       [99, 99, 99,  1,  6],
       [ 1,  5,  2,  5,  6],
       [ 8,  6,  5,  7,  4],
       [99, 99, 99,  8,  2],
       [99, 99, 99,  5,  7],
       [99, 99, 99,  9,  8],
       [99, 99, 99,  1,  1],
       [ 5,  8,  8,  8,  7],
       [ 4,  3,  3,  7,  3]])

In [3]:
## Testing the dropout for output layer
l = Layer(10, dropout_percentile = 0.5, activation_function = None)
l.w = np.random.randint(0, 10, (10, 5))

# Fix variables to 0
l.b = np.zeros((1,10))
l.b_moment1 = np.zeros_like(l.b)
l.b_moment2 = np.zeros_like(l.b)
l.w_moment1 = np.zeros_like(l.w)
l.w_moment2 = np.zeros_like(l.w)

previous = range(l.w.shape[1])

l.dropout(previous)
w_test = l.w
l.w = 99*np.ones_like(w_test)
l.reset_matrix(previous)
l.w

array([[99, 99, 99, 99, 99],
       [ 5,  2,  9,  4,  5],
       [ 8,  0,  0,  2,  7],
       [99, 99, 99, 99, 99],
       [99, 99, 99, 99, 99],
       [99, 99, 99, 99, 99],
       [99, 99, 99, 99, 99],
       [ 2,  1,  9,  9,  1],
       [ 9,  4,  8,  9,  8],
       [ 1,  1,  9,  3,  6]])

In [4]:
## Testing NN with dropout
NN = NeuralNetwork(learning_rate=0.01,max_epochs=1000,type_loss="mse")
NN.add(Layer(3)) # Add input layer
NN.add(Layer(6, dropout_percentile=0.5, activation_function=ActivationLayer("ReLu"))) # Add hidden layer
NN.add(Layer(4, dropout_percentile=0.5, activation_function=ActivationLayer("sigmoid"))) # Add output layer
NN.build_network()

indices = NN.dropout_matrices()

NN.layers[1].w = 99*np.ones_like(NN.layers[1].w)
NN.layers[2].w = 99*np.ones_like(NN.layers[2].w)

NN.restore_matrices()

# Check the imposition of 99 in the elements of the weight matrix
NN.layers[1].w

array([[ 3.77252549e-01, -1.11498686e-01, -2.68763766e-02,
        -3.45194354e-01,  2.19329474e-01, -6.59897834e-02],
       [-1.54791755e-01, -2.18744074e-01,  2.99897116e-01,
        -4.65630969e-03,  4.05454868e-01, -1.61827405e-01],
       [ 7.86263331e-02, -5.19212947e-02,  9.90000000e+01,
         9.90000000e+01,  4.84052671e-01,  9.90000000e+01]])

## Activation function

In [5]:
## Test softmax
x = np.array([8,5,0])

al = ActivationLayer(type = "softmax")
al.activation_function(x)

array([9.52269826e-01, 4.74107229e-02, 3.19450938e-04], dtype=float128)

## Test on regression

In [6]:
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Test the regression with the make_blobs with 4 features
X, y = make_blobs( n_samples=10000, n_features=4, centers=((0,0,0,0), (1,1,1,1)))
X = StandardScaler().fit_transform(X)
y = np.reshape(y,(-1,1))

# Split train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80)

In [7]:
# Build and train the neural network without dropout
NN = NeuralNetwork(learning_rate=0.01,max_epochs=100,type_loss="mse")
NN.add(Layer(4)) # Add input layer
NN.add(Layer(20, dropout_percentile=1, activation_function=ActivationLayer("ReLu"))) # Add hidden layer
NN.add(Layer(1, dropout_percentile=1, activation_function=ActivationLayer("sigmoid"))) # Add output layer
NN.build_network()

L = NN.fit_batch(X_train,y_train,toll = 0.01)
y_pred = NN.predict(X_test)

# Evaluate correcy preditions
y_pred[y_pred > 0.5] = 1
y_pred[y_pred < 0.5] = 0
print("Percentage of correct predictions: %.2f "%float(sum(y_pred==y_test)/len(y_pred)))

Percentage of correct predictions: 0.83 


With dropout:

In [8]:
NN = NeuralNetwork(learning_rate=0.01,max_epochs=100,type_loss="mse")
NN.add(Layer(4)) # Add input layer
NN.add(Layer(20, dropout_percentile=0.4, activation_function=ActivationLayer("sigmoid"))) # Add hidden layer
NN.add(Layer(1, dropout_percentile=0.3, activation_function=ActivationLayer("sigmoid"))) # Add output layer
NN.build_network()

L = NN.fit_batch(X_train,y_train,toll = 0.1)
y_pred = NN.predict(X_test)

# Evaluate correcy preditions
y_pred[y_pred > 0.5] = 1
y_pred[y_pred < 0.5] = 0
print("Percentage of correct predictions with dropout: %.2f "%float(sum(y_pred==y_test)/len(y_pred)))

Percentage of correct predictions with dropout: 0.80 
